In [1]:
from bs4 import BeautifulSoup
import requests
import re
import collections
import pickle

In [2]:
# init dict
Data = collections.defaultdict(list)

In [3]:
URL = 'https://a-z-animals.com/animals/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Finds all links
Animals = soup.find_all('a')

# Create a list with links
AnimalsA_Zpages = [pages.get('href') for pages in Animals 
                   if pages.get('href') != None
                   if pages.get('href').startswith('https://a-z-animals.com')]

In [4]:
# Removes references in text
ReferenceRemover = '\[\d*\]'

for AnimalPage in AnimalsA_Zpages[0:50]:
    
    # Open the page
    page = requests.get(AnimalPage, timeout=5)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    for Tags in soup.find_all('h2'):

        # Get the chapters
        try:
            Chapter = Tags['id']
        # skip other stuff
        except:
            continue

        # Finds descriptions or similar
        if Chapter.endswith('appearance'):
            
            '''
            # Get species name
            Species = soup.title\
                            .string\
                            .split(' Animal')[0]\
                            .lower()\
                            .capitalize()
            '''
            
            # Get species name
            Species = soup.find('h1').text\
                                        .lower()\
                                        .capitalize()


            # Get the next sibling (text)
            for Text in Tags.find_next_siblings('p'):

                # Add description data to dict
                if Chapter in Text.find_previous_siblings('h2')[0]['id']:
                    # Remove source
                    Paragraph = re.sub(ReferenceRemover, '', Text.text)
                    # Add to dict
                    Data[Species].append(tuple([1, Paragraph]))
                    
                    # Split into Sentences
                    #SentenceList = Paragraph.split('. ')
                    # Add to the dict
                    #Data[Species] += [(1, Sentence) for Sentence in SentenceList]
          
                # Add non description data to dict
                elif Chapter not in Text.find_previous_siblings('h2')[0]['id']:
                    # Remove source
                    Paragraph = re.sub(ReferenceRemover, '', Text.text)
                    # Add to dict
                    Data[Species].append(tuple([0, Paragraph]))
                    
                    # Split into Sentences
                    #SentenceList = Paragraph.split('. ')
                    # Add to the dict
                    #Data[Species] += [(0, Sentence) for Sentence in SentenceList]

In [5]:
with open('../data/processed/dataAZ_withMeasurements.pkl', 'wb') as f:
    pickle.dump(Data, f)

In [5]:
len(Data.keys())

12

In [7]:
Data['Alligator']

[(1,
  'Alligators are very large reptiles, with males growing up to 4.5 meters in length. The female Alligator tends to be slightly smaller, with a total body and tail length of between 3 and 3.5 meters. The Chinese Alligator is a much smaller species, almost half the size of a female American Alligator. Alligators have an armour-plated body that varies in colour from yellow, to green, to brown, finally turning almost completely black in old age. The tail of the Alligator is incredibly muscular and is used to propel the animal when it is in the water. Alligators have short, stocky legs with webbing between their toes. This not only helps them when they are swimming but also means that they can negotiate the muddy river banks with ease.'),
 (0,
  'The American Alligators are found in the south-eastern USA, throughout all of Florida and Louisiana, the southern parts of Georgia, Alabama and Mississippi, coastal South and North Carolina, eastern Texas, the south-eastern corner of Oklahoma